## 1. Environment Setup
Import libraries for neuroimaging (Nilearn), machine learning (Scikit-learn), and data handling.

In [ ]:
# Import libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nilearn import image, plotting, decoding
from nilearn.glm.first_level import make_first_level_design_matrix
from sklearn.model_selection import LeaveOneGroupOut
from nilearn.input_data import NiftiMasker

## 2. Configuration
Define dataset paths and parameters. We will focus on a single subject (e.g., `sub-01`) for this demonstration, but the logic applies to all.

In [ ]:
# Configuration
base_path = "data/ds101_R2.0.0/"
subject = "sub-01"
TR = 2.0
runs = [1, 2]

print(f"Analyzing Subject: {subject}")

## 3. Data Preparation for ML
Unlike standard GLM which models the whole time series, for decoding we often want **one sample per trial** (or per block).

**Strategy:**
1. Load the fMRI time series.
2. Load the events and identify the onset of each trial.
3. Extract the specific fMRI volume (scan) corresponding to each trial onset (accounting for hemodynamic lag, approx 4-6s).
4. Create a list of `X` (brain images) and `y` (labels: congruent/incongruent).

In [ ]:
# Helper function to map trial types
def map_congruency(tt):
    tt = str(tt).lower()
    if "incongruent" in tt:
        return "incongruent"
    elif "congruent" in tt:
        return "congruent"
    else:
        return None

# Lists to hold our dataset
conditions = []
run_labels = []
fmri_volumes = []

# We need a mask to extract data from the brain only
# We'll compute a simple mask from the first run
mask_img = decoding.compute_mask(os.path.join(base_path, subject, "func", f"{subject}_task-simon_run-1_bold.nii.gz"))

print("Extracting trial data...")

for run in runs:
    # 1. Load fMRI image
    fmri_file = os.path.join(base_path, subject, "func", f"{subject}_task-simon_run-{run}_bold.nii.gz")
    img = image.load_img(fmri_file)
    
    # 2. Load Events
    events_file = os.path.join(base_path, subject, "func", f"{subject}_task-simon_run-{run}_events.tsv")
    events = pd.read_csv(events_file, sep="\t")
    
    # Filter and map events
    events["condition"] = events["trial_type"].apply(map_congruency)
    events = events[events["condition"].notna()]
    
    # 3. Extract Volumes
    # We assume the peak BOLD response is ~5 seconds after onset.
    # TR = 2.0s, so we look approx 2-3 volumes after onset.
    # Simple approach: Take the volume closest to (onset + 5s)
    
    for _, row in events.iterrows():
        onset = row["onset"]
        target_time = onset + 5.0 # Hemodynamic delay
        target_vol_idx = int(np.round(target_time / TR))
        
        # Check bounds
        if target_vol_idx < img.shape[-1]:
            # Extract the single 3D volume
            vol = image.index_img(img, target_vol_idx)
            fmri_volumes.append(vol)
            conditions.append(row["condition"])
            run_labels.append(run)

print(f"Total trials extracted: {len(conditions)}")
print(f"Conditions distribution: {pd.Series(conditions).value_counts().to_dict()}")

## 4. Decoding Analysis (SVM)
We use `nilearn.decoding.Decoder` to train a Support Vector Machine (SVM).

**Setup:**
- **Estimator**: SVC (Support Vector Classification).
- **Cross-Validation**: Leave-One-Run-Out (Train on Run 1, Test on Run 2, and vice versa).
- **Metric**: Accuracy.

In [ ]:
# Initialize the Decoder
decoder = decoding.Decoder(
    estimator='svc',           # Support Vector Classifier
    mask=mask_img,             # Brain mask
    standardize=True,          # Z-score normalization
    scoring='accuracy',        # Evaluation metric
    cv=LeaveOneGroupOut(),     # Cross-validation strategy
    n_jobs=-1                  # Use all CPUs
)

# Fit the model
# X = List of 3D images (fmri_volumes)
# y = List of labels (conditions)
# groups = List of run IDs (run_labels) for CV splitting

print("Training SVM classifier...")
decoder.fit(fmri_volumes, conditions, groups=run_labels)

# Print results
print("-" * 30)
print(f"Classification Accuracy: {np.mean(decoder.cv_scores_['congruent']):.2f}")
print("-" * 30)
print("Chance level is 0.50 (50%)")

## 5. Visualization of Weight Maps
The decoder produces a "weight map" (coefficients) indicating which voxels contributed most to the decision boundary.
- **Positive weights**: Associated with one class (e.g., Incongruent).
- **Negative weights**: Associated with the other class (e.g., Congruent).

In [ ]:
# Retrieve the weight map (coefficients)
# The decoder stores this as a Nifti image
weight_img = decoder.coef_img_["incongruent"]

# Plot the weights
plotting.plot_stat_map(
    weight_img, 
    bg_img=image.mean_img(fmri_volumes[0]), 
    title=f"SVM Weights: {subject} (Incongruent vs Congruent)",
    display_mode='z',
    cut_coords=[-10, 0, 10, 20, 30]
)
plotting.show()

## 6. Conclusion
If the accuracy is significantly above 0.50, it suggests that the spatial pattern of brain activity contains information distinguishing the two cognitive states.